In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

data_dir = 'data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)


def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()



inputs, classes = next(iter(dataloaders['train']))


out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)


        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)


                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)


                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()


                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


    model.load_state_dict(best_model_wts)
    return model




model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)



step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)


model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)


exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

FileNotFoundError: [Errno 2] No such file or directory: 'data/hymenoptera_data/train'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/My Drive/Datasets/my_file.csv'
df = pd.read_csv(file_path)
print(df.head())


In [7]:
import os
import pandas as pd

# Define paths to your folders
train_ants_dir = '/content/drive/MyDrive/hymenoptera_data/train/ants'
train_bees_dir = '/content/drive/MyDrive/hymenoptera_data/train/bees'
val_ants_dir = '/content/drive/MyDrive/hymenoptera_data/val/ants'
val_bees_dir = '/content/drive/MyDrive/hymenoptera_data/val/bees'

# Initialize lists to store file paths and labels
file_paths = []
labels = []
dataset_type = []

# Helper function to collect file paths and labels
def collect_file_paths(folder_path, label, dataset):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):  # Check if it's a file
            file_paths.append(file_path)
            labels.append(label)
            dataset_type.append(dataset)

# Collect file paths for training data
collect_file_paths(train_ants_dir, 'ant', 'train')
collect_file_paths(train_bees_dir, 'bee', 'train')

# Collect file paths for validation data
collect_file_paths(val_ants_dir, 'ant', 'val')
collect_file_paths(val_bees_dir, 'bee', 'val')

# Create the DataFrame
df = pd.DataFrame({
    'file_path': file_paths,
    'label': labels,
    'dataset': dataset_type
})

# Display the DataFrame
pd.set_option('display.max_colwidth', None)
print(df.head())

# Verify the dataset type counts
print(df['dataset'].value_counts())


                                                                          file_path  \
0       /content/drive/MyDrive/hymenoptera_data/train/ants/382971067_0bfd33afe0.jpg   
1        /content/drive/MyDrive/hymenoptera_data/train/ants/36439863_0bec9f554f.jpg   
2                  /content/drive/MyDrive/hymenoptera_data/train/ants/175998972.jpg   
3  /content/drive/MyDrive/hymenoptera_data/train/ants/Nepenthes_rafflesiana_ant.jpg   
4       /content/drive/MyDrive/hymenoptera_data/train/ants/342438950_a3da61deab.jpg   

  label dataset  
0   ant   train  
1   ant   train  
2   ant   train  
3   ant   train  
4   ant   train  
dataset
train    245
val      153
Name: count, dtype: int64


In [8]:
print(df.groupby(['dataset', 'label']).size())


dataset  label
train    ant      124
         bee      121
val      ant       70
         bee       83
dtype: int64
